# Simulation
Probability theory has an infamous inception for its association with gambling. Monte Carlo, the European casino capital, is another unfortunate presence. However, naming it Macau simulation or Hong Kong Jockey Club simulation does not make me feel any better. I decide to simply call it "simulation".

Simulation has been widely used for (i) checking finite-sample performance of asymptotic theory; (ii) bootstrap, an automated inference procedure; (iii) generating non-standard distributions; (iv) approximating integrals with no analytic expressions. In this lecture, we will focus on (i) and (ii), whereas (iii) and (iv) will be deferred to the next lecture on integration.
From now on, we will start to write script. A script is a piece of code for a particular purpose. We do not write a script of thousands of lines from the beginning to the end; we develop it recursively. We cut a big job into small manageable tasks. Write a small piece, test it, and perhaps encapsulate it into a user-defined function. Small pieces are integrated by the super structure. This is just like building an Airbus 380. The engines and wings are made in UK, the fuselage is made in Germany and so on. All pieces are assembled in Toulouse, France, and then the giant steel bird can fly. Finally, add comments to the script to facilitate readability. Without comments you will forget what you did when you open the script again one month later.

**Example**

Zu Chongzhi (429--500 AD), an ancient Chinese mathematician, calculated $\pi$ being between 3.1415926 and 3.1415927, which for 900 years held the world record of the most accurate $\pi$. He used a deterministic approximation algorithm. Now imagine that we present to Zu Chongzhi, with full respect and admiration, a modern PC. How can he achieve a better approximation? Of course, we suppose that he would not google it.

Standing on the shoulder of laws of large numbers, $\pi$ can be approximated by stochastic algorithm.


In [1]:
import numpy as np
n = 10000
Z = np.random.uniform(low = 0.0, high = 1.0, size = (n // 2, 2)) # uniform distribution ranging (0,1)
X = Z - np.full((n // 2, 2), 0.5)
Y = (X * X).sum(axis = 1)
W = Y ** (1 / 2.0)
inside = len([i for i in W if i <= 0.5]) / len(W) # the center of the cirle is (0.5,0.5)
pi_hat = 4 * inside # the area of a circle = pi * r^2
print(pi_hat)

3.1344


## Finite Sample Evaluation
In the real world, a sample is finite. The distribution of a statistic in finite sample depends on the sample size $n$, which has simple a mathematical expression only in rare cases. Fortunately, the expression can often be simplified when we imagine the sample size being arbitrarily large. Asymptotic theory is such apparatus to approximate finite sample distributions. It is so far the best mathematical tool that helps us understand the behavior of estimators and tests, either in econometrics or in statistics in general. Simulation is one way to evaluate the accuracy of approximation.

Even though real data empirical example can also be used to illustrate a statistical procedure, artificial data are convenient and boast advantages. The prevalent paradigm in statistics is to assume that the data are generated from a model. We, as researchers, check how close the estimate is to the model, which is often characterized by a set of unknown parameters. In simulation we have full control of the data generation process, so we also know the true parameter. In a real example, however, we have no knowledge about the true model, so we cannot directly evaluate the quality of parameter estimation.

(It would be a different story if we are mostly interested in prediction, as we often encounter in machine learning. In such cases, we can split the data into two parts: one part for modeling and estimation, and the other for verification.)

**Example**

In OLS theory, the classical approach is to view $X$ as fixed regressions, and only cares about the randomness of the error term. Modern econometrics textbook emphasizes that a random $X$ is more appropriate for econometrics applications. In rigorous textbooks, the moment of $X$ is explicitly stated. Is asymptotic inferential theory for the OLS estimator---consistency and asymptotic normality---valid when $X$ follows a __[Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution)__ with shape coefficient 1.5? (A Pareto distribution with shape coefficient between 1 and 2 has finite mean but infinite variance.)

1. given sample size, get OLS b_hat and its associated t_value.
2. wrap t_value as a user-defined function so that we can replicate for many times.
3. given sample size, report the size under two distributions.
4. wrap it again as a user-defined function, ready for different sample sizes.
5. develop the super structure.
6. add comments and documentation.


In [2]:
import numpy as np
from scipy import stats
import datetime


np.random.seed(888)
#set the parameters
Rep = 1000
b0 = np.mat([[1], [1]])
df = 1 #t dist. with df = 1 is Cauchy

#the workhorse functions
def simulation(n, type = 'Normal', df = df):
    #a function gives the t-value under the null
    if type == 'Normal':
        e = np.random.normal(size = (n, 1))
    elif type == 'T':
        e = np.random.standard_t(df, size = (n, 1))

    X = np.column_stack((np.full((n, 1), 1), np.random.pareto(1.5, size = (n, 1))))
    Y = np.dot(X, b0) + e

    del e

    bhat = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, Y))
    bhat2 = bhat[1]  #parameter we want to test

    e_hat = Y - np.dot(X, bhat)
    sigma_hat_square = np.dot(e_hat.T, e_hat) / (n - 2)
    sig_B = np.linalg.inv(np.dot(X.T, X)) * np.array(sigma_hat_square)
    t_value_2 = (bhat2 - b0[1, 0]) / (sig_B[1, 1] ** (1 / 2.0))

    out = {'bhat2': bhat2, 't_value': t_value_2}
    return out

#report the empirical test size
def report(n):
    # collect the test size from the two distributions
    # this function contains some repetitive code, but is OK for such a simple one
    TEST_SIZE = [0, 0, 0]

    # e ~ normal distribution, under which the t-dist is exact
    normalRes = {'bhat2':([0] * Rep), 't_value':([0] * Rep)}

    # e ~ t-distribution, under which the exact distribution is complicated.
    # we rely on asymptotic normal distribution for inference instead
    cauchyRes = {'bhat2':([0] * Rep), 't_value':([0] * Rep)}
    for i in range(Rep):
        normalRes['bhat2'][i] = simulation(n, 'Normal')['bhat2']
        normalRes['t_value'][i] = simulation(n, 'Normal')['t_value']
        cauchyRes['bhat2'][i] = simulation(n, 'T', df)['bhat2']
        cauchyRes['t_value'][i] = simulation(n, 'T', df)['t_value']


    TEST_SIZE[0] = len([normalRes['t_value'][i][0][0] for normalRes['t_value'][i] in normalRes['t_value'] if abs(normalRes['t_value'][i][0][0]) > stats.t.ppf(q=0.975, df=n - 2)]) / float(len(normalRes['t_value']))
    TEST_SIZE[1] = len([normalRes['t_value'][i][0][0] for normalRes['t_value'][i] in normalRes['t_value'] if abs(normalRes['t_value'][i][0][0]) > stats.norm.ppf(q=0.975)]) / float(len(normalRes['t_value']))
    TEST_SIZE[2] = len([cauchyRes['t_value'][i][0][0] for cauchyRes['t_value'][i] in cauchyRes['t_value'] if abs(cauchyRes['t_value'][i][0][0]) > stats.norm.ppf(q=0.975)]) / float(len(cauchyRes['t_value']))

    return TEST_SIZE

if __name__ == "__main__":
    pts0 = datetime.datetime.now()
    # run the calculation of the empirical sizes for different sample sizes
    NN = np.array([5, 10, 200, 5000])
    RES = {'exact':([0] * (len(NN))), 'normal.asym':([0] * (len(NN))), 'cauchy.asym':([0] * (len(NN)))}
    for i in range(len(NN)):
        RES['exact'][i] = report(NN[i])[0]
        RES['normal.asym'][i] = report(NN[i])[1]
        RES['cauchy.asym'][i] = report(NN[i])[2]
    print(RES)
    print(datetime.datetime.now() - pts0)


{'exact': [0.045, 0.06, 0.04, 0.043], 'normal.asym': [0.155, 0.08, 0.049, 0.044], 'cauchy.asym': [0.158, 0.098, 0.056, 0.023]}
0:00:40.657456


When we look at the table, we witness that the distribution of $X$ indeed does not matter. Why? $$ \sqrt{n} (\hat{\beta} - \beta_0) = (X'X/n)^{-1} (X'e/\sqrt{n}) $$ Even though $X'X/n$ does not converge, the self-normalized $t$ statistic is immune of the problem. What matters is the distribution of the error term. The first column is the when the error is normal, and we use the exactly distribution theory to find the critical value (according to the $t$ distribution.) The second column still uses the normal distribution in the error term, but change the critical value to be from the normal distribution, which is based on asymptotic approximation. When sample size is small, obvious size distortion is observed; but the deviation is mitigated when the sample size increases. When the error distribution is Cauchy, the so called "exact distribution" is not longer exact--- it is exact only if the error is normal and independent from $x$. If we attempt to use the asymptotic normal approximation, we find that the asymptotic approximation breaks down. The test size does not converge to the nominal 5% as the sample size increases.

## Bootstrap
Bootstrap, originated from Efron (1979), is an extremely powerful and influential idea for estimation and inference.

Let $X_1, X_2, \ldots, X_n \sim F$ be an i.i.d. sample of $n$ observations following a distribution $F$. The finite sample distribution of a statistic $T_n(\theta)\sim G_n(\cdot, F)$ usually depends on the sample size $n$, as well as the known true distribution $F$. Asymptotic theory approximates $G_n(\cdot, F)$ by its limit $$G(\cdot, F) := \lim_{n\to\infty} G_n(\cdot, F);$$ if $T_n(\theta)$ is asymptotically pivotal, then $G_n(\cdot, F)$ is independent of $F$.

Instead of referring to the limiting distribution, Bootstrap replaces the unknown distribution $F$ in $G_n(\cdot, F)$ by a consistent estimator $F_n$ of the true distribution, for example, the empirical distribution function. Bootstrap inference is drawn from the bootstrap distribution $$ G^{*}_n(\cdot):= G_n(\cdot, F_n) $$

Implementation of bootstrap is almost always a Monte Carlo simulation. In i.i.d. environment we sample over each observation with equal weight, while in dependent dataset such as time series, clustering data or networks, we must adjust the sampling schedule to preserve the dependence structure.

In many regular cases, it is possible to show in theory the consistency of bootstrap: the statistic of interest and its bootstrap version converge to the same asymptotic distribution, or $G^*_n(a)\to G(a)$ for $a$ such that $G(a)$ is continuous. However, bootstrap consistency can fail when the distribution of the statistic is discontinuous in the limit. Bootstrap is invalid in such cases. For instance, naive bootstrap fails to replicate the asymptotic distribution of the two-stage least squares estimator under weak instruments.

### Bootstrap Estimation
Bootstrap is simple enough to be done by a ply-family function for repeated simulations. Alternatively, R package boot provides a general function boot().

Bootstrap is useful when the analytic formula of the variance of an econometric estimator is too complex to derive or code up.

#### Example
One of the most popular estimators for a sample selection model is Heckman(1979)'s two-step method. Let the outcome equation be $$y_i = x_i \beta + u_i$$ and the selection equation be $$D_i = z_i \gamma + v_i$$ To obtain a point estimator, we simply run a Probit in the selection model, predict the probability of participation, and then run an OLS of $y_i$ on $x_i$ and $\lambda (\hat{D}_i)$ in the outcome model, where $\lambda(\cdot)$ is the inverse Mill's ratio. However, as we can see from Heckman(1979)'s original paper, the asymptotic variance expression of the two-step estimator is very complicated. Instead of following the analytic formula, we bootstrap the variance.

In [3]:
import rpy2
from rpy2.robjects import r
r('''install.packages('sampleSelection')''')
r('''install.packages('lmtest')''')
r('''library(sampleSelection)''')
r('''library(lmtest)''')
r('''data(Mroz87)''')
r('''selection_eq = lfp ~age + faminc + exper + educ''')
r('''outcome_eq = wage ~ exper + educ''')
r('''heck = heckit(selection_eq, outcome_eq, data = Mroz87)''')
heckCoefTest = r('''coeftest(heck)''')
print(heckCoefTest)



z test of coefficients:



                 Estimate  Std. Error z value  Pr(>|z|)    

(Intercept)   -1.1385e-01  4.0442e-01 -0.2815  0.778319    

age           -3.6696e-02  6.7110e-03 -5.4680 4.552e-08 ***

faminc         1.0319e-05  4.3393e-06  2.3780  0.017409 *  

exper          7.4511e-02  7.2030e-03 10.3444 < 2.2e-16 ***

educ           6.8872e-02  2.3978e-02  2.8723  0.004075 ** 

(Intercept)   -1.9500e+00  1.8023e+00 -1.0819  0.279277    

exper          1.6062e-02  3.3892e-02  0.4739  0.635554    

educ           4.8147e-01  8.2271e-02  5.8523 4.848e-09 ***

invMillsRatio -3.0324e-01  9.8862e-01 -0.3067  0.759051    

sigma          3.1080e+00          NA      NA        NA    

rho           -9.7565e-02          NA      NA        NA    

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1





Below is the function for a single bootstrap. For convenience, I keep using heckit but only save the point estimates.

In [4]:
import pandas as pd
import statistics
from rpy2.robjects import r
def boot_heck():
    r('''library(sampleSelection)''')
    r('''library(lmtest)''')
    r('''data(Mroz87)''')
    r('''selection_eq = lfp ~age + faminc + exper + educ''')
    r('''outcome_eq = wage ~ exper + educ''')
    r('''n = nrow(Mroz87);
    indices = sample(1:n, n, replace = T);
    Mroz87_b = Mroz87[indices,]
    ''')
    heck_b = r('''coef(heckit(selection_eq, outcome_eq, data = Mroz87_b))''')
    return (heck_b)


Implementation is just a repeated evaluation.

In [5]:
boot_Rep = 199
column_names = ['selection_intercept', 'age', 'faminc', 'selection_exper', 'selection_educ', 'outcome_intercept', 'outcome_exper', 'outcome_educ', 'invMillsRatio', 'sigma', 'rho']
Heck_B = pd.DataFrame(index = range(boot_Rep), columns = column_names)
for i in range(boot_Rep):
    Heck_B.iloc[i,:] = boot_heck()
Heck_b_sd = pd.DataFrame(index = ['sd'], columns = column_names)
for i in range(len(column_names)):
    Heck_b_sd.iloc[:, i] = statistics.stdev(Heck_B.iloc[:,i])
print(Heck_b_sd)

    selection_intercept         age       faminc selection_exper  \
sd             0.410106  0.00702412  4.73325e-06      0.00784671   

   selection_educ outcome_intercept outcome_exper outcome_educ invMillsRatio  \
sd      0.0248566            2.4349     0.0422959    0.0986313       1.55759   

      sigma       rho  
sd  0.40795  0.448673  


### Bootstrap Test
Bootstrap is particularly helpful in inference. Indeed, we can rigorously prove that bootstrap enjoys high-order improvement relative to asymptotic approximation if the test statistic is asymptotically pivotal. Loosely speaking, if the test statistic is asymptotically pivotal, a bootstrap hypothesis testing can be more accurate than its analytic asymptotic counterpart.
#### Example
We use bootstrap to test a hypothesis about the population mean. The test is carried out via by a $t$-statistic. The distribution of the sample is either normal or zero-centered chi-square. It will show that the bootstrap test size is more precise than that of the asymptotic approximation.

We first prepare the workhorse functions.

In [6]:
import statistics
import numpy as np
import pandas as pd
from scipy import stats

#the t-statistic for a null hypothesis mu
def T_stat(Y, mu, n):
    t_stat = (n ** (1 / 2.0)) * (statistics.mean(Y) - mu) / statistics.stdev(Y)
    return t_stat

#the bootstrap function
def boot_test(Y, boot_Rep, alpha, n, mu):
    #INPUT Y: the sample boot_Rep: number of bootstrap replications
    lenY = len(Y)
    boot_T = [0] * boot_Rep

    #bootstrap in action
    for r in range(boot_Rep):
        indices = np.random.choice(range(lenY), size = lenY, replace = True) #resampling the index
        resampled_Y = [Y[i] for i in indices] #construct a bootstrap artificial sample
        boot_T[r] = abs(T_stat(resampled_Y, statistics.mean(Y), n))
        #the bootstrapped t-statistic mu is replaced by 'mean(Y)' to mimic the situation under the null

    #bootstrap critical value
    boot_critical_value = (pd.DataFrame(boot_T)).quantile(1 - alpha)[0]

    #bootstrap test decision
    return int(abs(T_stat(Y, mu, n)) > boot_critical_value)


A key point for bootstrap test is that the null hypothesis must be imposed no matter the hypothesized parameter is true value or not. Therefore the bootstrap t-statistic is $$ T^{}_{n} = \frac{\bar{X^{}} - \bar{X}} { s^{*} / \sqrt{n} }. $$ That is, the bootstrap $t$-statistic is centered at $\bar{X}$, the sample mean of $F_n$, rather than $\theta$, the population mean of $F$. This is because in the bootstrap world the ``true'' distribution is $F_n$. If we wrongly center the bootstrap t-statistic at $\theta$, then the test will have no power when the null hypothesis is false.

The following chuck of code report the rejection probability from three decision rules.

In [7]:
def compare(distribution, boot_Rep, alpha, n, mu):
    #this function generates a sample of n observations and it returns the testing results from three decision rules
    if distribution == 'normal':
        X = np.random.normal(size =(n,))
    elif distribution == 'chisq':
        X = np.random.chisquare(df = 3, size = n) - 3

    t_value_X = T_stat(X, mu, n) #T-statistic

    #compare it to the 97.5% of t-distribution
    exact = int(abs(t_value_X) > stats.t.ppf(q = 0.975, df = n - 1))

    #compare it to the 97.5% of normal distribution
    asym = int(abs(t_value_X) > 1.96)

    #decision from bootstrap
    boot_decision = boot_test(X, boot_Rep, alpha, n, mu)

    return [exact, asym, boot_decision]

Here the nominal size of the test is 5%. The program reports the empirical size ---the ratio between the number of rejections to the total number of replications. The closer is the empirical size to the nominal size, the more accurate is the test. We find here the bootstrap test is better than the asymptotic test.

When the underlying distribution is a $\chi^2$, there is no explicit exact distribution. However, we can still compare the asymptotic size with the bootstrap size.

In [8]:
if __name__ == "__main__":
    n = 20
    distribution = 'normal'
    boot_Rep = 199
    MC_rep = 2000
    alpha = 0.05
    mu = 0

    np.random.seed(111)

    #Monte Carlo simulation and report the rejection probability
    column_name = ['exact', 'asym', 'bootstrap']
    norm_res = pd.DataFrame(index = range(MC_rep + 1), columns = column_name)
    for i in range(MC_rep + 1):
        norm_res.iloc[i, :] = compare(distribution, boot_Rep, alpha, n, mu)
        if i == MC_rep:
            for j in range(3):
                norm_res.iloc[MC_rep, j] = (norm_res.iloc[:MC_rep, j]).mean()
    print(norm_res.iloc[MC_rep, :])

    distribution = 'chisq'
    np.random.seed(666)

    #Simulate and report the rejection probability
    chisq_res = pd.DataFrame(index = range(MC_rep + 1), columns = column_name)
    for i in range(MC_rep + 1):
        chisq_res.iloc[i, :] = compare(distribution, boot_Rep, alpha, n, mu)
        if i == MC_rep:
            for j in range(3):
                chisq_res.iloc[MC_rep, j] = (chisq_res.iloc[:MC_rep, j]).mean()
    print(chisq_res.iloc[MC_rep, :])

exact        0.0440
asym         0.0620
bootstrap    0.0465
Name: 2000, dtype: float64
exact        0.0740
asym         0.0900
bootstrap    0.0655
Name: 2000, dtype: float64


Again, here the "exact test" is no longer exact. The asymptotic test works fairly reasonable, while the bootstrap is closer to the nominal size.